In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.decomposition import PCA

from sklearn.metrics import classification_report,precision_score
from sklearn.metrics import confusion_matrix,roc_curve,accuracy_score,f1_score,recall_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss



from sklearn.datasets import make_classification
import numpy as np


# import function to perform feature selection
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('cleanquestions.csv')

In [3]:
df

,is_duplicate,question1_len,question2_len,common_words,cosine_distance,euclidean_distances,minkowski_distances,manhattan_distances,jaccard_distances,fuzz_ratio,fuzz_partial_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio
0,0,41,35,5,0.977118,0.213924,0.212742,0.261590,0.916667,92,100,100,92
1,0,31,67,4,0.708126,0.764034,0.550300,2.283288,0.900000,57,90,89,59
2,0,42,32,3,0.599502,0.894983,0.708720,2.040962,0.909091,27,51,84,68
3,0,24,21,0,0.000000,1.414214,1.086520,3.351458,0.857143,18,8,27,27
4,0,60,29,2,0.249909,1.224819,0.838590,4.164808,0.909091,25,52,51,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994,0,32,32,2,0.604812,0.889031,0.722682,1.812016,0.857143,69,71,72,62
3995,0,45,34,2,0.306822,1.177436,0.805822,3.889226,0.923077,25,39,46,41
3996,0,14,17,0,0.000000,1.414214,1.101967,3.095291,0.833333,26,29,39,39
3997,0,34,33,1,0.190946,1.272049,1.026242,2.646534,0.857143,72,73,66,66


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   is_duplicate           3999 non-null   int64  
 1   question1_len          3999 non-null   int64  
 2   question2_len          3999 non-null   int64  
 3   common_words           3999 non-null   int64  
 4   cosine_distance        3983 non-null   float64
 5   euclidean_distances    3999 non-null   float64
 6   minkowski_distances    3999 non-null   float64
 7   manhattan_distances    3999 non-null   float64
 8   jaccard_distances      3999 non-null   float64
 9   fuzz_ratio             3999 non-null   int64  
 10  fuzz_partial_ratio     3999 non-null   int64  
 11  fuzz_token_set_ratio   3999 non-null   int64  
 12  fuzz_token_sort_ratio  3999 non-null   int64  
dtypes: float64(5), int64(8)
memory usage: 406.3 KB


In [5]:
df.describe()

,is_duplicate,question1_len,question2_len,common_words,cosine_distance,euclidean_distances,minkowski_distances,manhattan_distances,jaccard_distances,fuzz_ratio,fuzz_partial_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio
count,3999.000000,3999.000000,3999.000000,3999.000000,3983.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000
mean,0.378595,35.412603,35.493123,2.639160,0.572696,0.847606,0.660080,2.112683,0.818872,59.830708,69.227057,75.029507,63.167542
std,0.485098,19.745575,21.283829,1.640708,0.284732,0.369850,0.254338,1.417931,0.244145,23.129658,21.610529,23.489966,21.669350
min,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,22.000000,21.000000,2.000000,0.386632,0.637328,0.563056,1.011302,0.857143,41.000000,53.000000,61.000000,48.000000
50%,0.000000,31.000000,30.000000,3.000000,0.604054,0.891609,0.705613,1.881930,0.888889,60.000000,69.000000,80.000000,64.000000
75%,1.000000,44.000000,43.000000,3.000000,0.797229,1.107134,0.817132,3.031962,0.916667,78.000000,86.000000,100.000000,80.000000
max,1.000000,257.000000,196.000000,13.000000,1.000000,1.414214,1.259921,7.401956,0.970588,100.000000,100.000000,100.000000,100.000000


In [6]:
df.isnull().sum()

is_duplicate              0
question1_len             0
question2_len             0
common_words              0
cosine_distance          16
euclidean_distances       0
minkowski_distances       0
manhattan_distances       0
jaccard_distances         0
fuzz_ratio                0
fuzz_partial_ratio        0
fuzz_token_set_ratio      0
fuzz_token_sort_ratio     0
dtype: int64

In [7]:
# fill null values with mean
df.fillna(df.mean(), inplace=True)

In [8]:
for i in df.columns:
    print(i)
    print(df[i].value_counts())
    print('******')
    print('')

is_duplicate
0    2485
1    1514
Name: is_duplicate, dtype: int64
******

question1_len
25     131
26     120
22     115
31     113
20     112
      ... 
116      1
178      1
124      1
157      1
140      1
Name: question1_len, Length: 120, dtype: int64
******

question2_len
21     140
23     140
27     132
25     131
24     122
      ... 
110      1
170      1
131      1
118      1
176      1
Name: question2_len, Length: 133, dtype: int64
******

common_words
2     1130
3     1090
4      554
1      483
0      357
5      197
6       95
7       40
8       21
9       16
10       5
11       5
12       5
13       1
Name: common_words, dtype: int64
******

cosine_distance
0.000000    341
1.000000    218
1.000000     94
0.572696     16
0.689836      5
           ... 
0.341648      1
0.766150      1
0.700947      1
0.459752      1
0.190946      1
Name: cosine_distance, Length: 3312, dtype: int64
******

euclidean_distances
0.000000e+00    281
1.414214e+00    247
1.414214e+00     93
1.000000

In [9]:
# split X and y
x = df.iloc[:, 1:]         # select all columns except the first one
y = df.iloc[:, 0]          # select only the first column

In [10]:
x.shape

(3999, 12)

In [11]:
y.shape

(3999,)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=45)

In [13]:
print("x_train shape", x_train.shape)
print("x_test shape", x_test.shape)
print("y_train shape", y_train.shape)
print("y_test shape", y_test.shape)

x_train shape (2799, 12)
x_test shape (1200, 12)
y_train shape (2799,)
y_test shape (1200,)


In [14]:


# create scaler object
scaler = MinMaxScaler()

# fit and transform data

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)


In [15]:
from lazypredict.Supervised import LazyClassifier

In [16]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(x_train_scaled, x_test_scaled, y_train, y_test)

print(models)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:51<00:00,  1.76s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
LGBMClassifier                     0.70               0.68     0.68      0.70   
QuadraticDiscriminantAnalysis      0.66               0.68     0.68      0.67   
NuSVC                              0.70               0.68     0.68      0.69   
RandomForestClassifier             0.70               0.68     0.68      0.70   
AdaBoostClassifier                 0.69               0.68     0.68      0.69   
SVC                                0.69               0.68     0.68      0.69   
Perceptron                         0.68               0.68     0.68      0.68   
XGBClassifier                      0.69               0.67     0.67      0.69   
ExtraTreesClassifier               0.70               0.67     0.67      0.69   
GaussianNB                         0.66               0.67     0.67      0.67   
BernoulliNB                 

In [17]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.70,0.68,0.68,0.70,7.58
QuadraticDiscriminantAnalysis,0.66,0.68,0.68,0.67,0.22
NuSVC,0.70,0.68,0.68,0.69,4.64
RandomForestClassifier,0.70,0.68,0.68,0.70,2.71
AdaBoostClassifier,0.69,0.68,0.68,0.69,1.11
SVC,0.69,0.68,0.68,0.69,2.90
Perceptron,0.68,0.68,0.68,0.68,0.07
XGBClassifier,0.69,0.67,0.67,0.69,19.46
ExtraTreesClassifier,0.70,0.67,0.67,0.69,1.71


# HYPERPARAMETER TUNING FOR LOGISTIC REGRESSION

In [18]:
# Define the hyperparameter grid to search over
param_grid = {'penalty': ['l1', 'l2', 'elasticnet', 'none'],
              'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
              'max_iter': [100,300, 500]}

In [19]:
# Create a logistic regression object
logreg = LogisticRegression()

# Create a GridSearchCV object to search over the hyperparameter grid
grid_search = GridSearchCV(logreg, param_grid=param_grid, cv=5, verbose=2)



In [20]:
# Fit the GridSearchCV object to the training data
grid_search.fit(x_train_scaled, y_train)


Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] END .........max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END .........max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END .........max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END .........max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END .........max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END .............max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .............max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .............max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .............max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .............max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........max_iter=100, penalty=l1, solver=liblinear; total time=   0.3s
[CV] END .........max_iter=100, penalty=l1, sol

[CV] END .........max_iter=300, penalty=l1, solver=liblinear; total time=   0.4s
[CV] END .........max_iter=300, penalty=l1, solver=liblinear; total time=   0.4s
[CV] END .........max_iter=300, penalty=l1, solver=liblinear; total time=   0.3s
[CV] END .........max_iter=300, penalty=l1, solver=liblinear; total time=   0.7s
[CV] END .........max_iter=300, penalty=l1, solver=liblinear; total time=   0.4s
[CV] END ...............max_iter=300, penalty=l1, solver=sag; total time=   0.0s
[CV] END ...............max_iter=300, penalty=l1, solver=sag; total time=   0.0s
[CV] END ...............max_iter=300, penalty=l1, solver=sag; total time=   0.0s
[CV] END ...............max_iter=300, penalty=l1, solver=sag; total time=   0.0s
[CV] END ...............max_iter=300, penalty=l1, solver=sag; total time=   0.0s
[CV] END ..............max_iter=300, penalty=l1, solver=saga; total time=   0.1s
[CV] END ..............max_iter=300, penalty=l1, solver=saga; total time=   0.0s
[CV] END ..............max_i

[CV] END .........max_iter=500, penalty=l1, solver=liblinear; total time=   0.3s
[CV] END .........max_iter=500, penalty=l1, solver=liblinear; total time=   0.6s
[CV] END .........max_iter=500, penalty=l1, solver=liblinear; total time=   0.4s
[CV] END ...............max_iter=500, penalty=l1, solver=sag; total time=   0.0s
[CV] END ...............max_iter=500, penalty=l1, solver=sag; total time=   0.0s
[CV] END ...............max_iter=500, penalty=l1, solver=sag; total time=   0.0s
[CV] END ...............max_iter=500, penalty=l1, solver=sag; total time=   0.0s
[CV] END ...............max_iter=500, penalty=l1, solver=sag; total time=   0.0s
[CV] END ..............max_iter=500, penalty=l1, solver=saga; total time=   0.1s
[CV] END ..............max_iter=500, penalty=l1, solver=saga; total time=   0.1s
[CV] END ..............max_iter=500, penalty=l1, solver=saga; total time=   0.1s
[CV] END ..............max_iter=500, penalty=l1, solver=saga; total time=   0.1s
[CV] END ..............max_i

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'max_iter': [100, 300, 500],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             verbose=2)

In [21]:

# Print the best hyperparameters found by GridSearchCV
print('Best hyperparameters:', grid_search.best_params_)


Best hyperparameters: {'max_iter': 300, 'penalty': 'none', 'solver': 'saga'}


In [22]:

# Use the best estimator found by GridSearchCV to predict the validation set
y_pred = grid_search.best_estimator_.predict(x_test_scaled)

In [23]:

# Calculate the accuracy of the logistic regression model on the validation set
accuracy = (y_pred == y_test).mean()
print('Validation set accuracy:', accuracy)

Validation set accuracy: 0.6658333333333334


In [25]:
print(confusion_matrix(y_test,y_pred))


[[546 179]
 [222 253]]


In [26]:

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.71      0.75      0.73       725
           1       0.59      0.53      0.56       475

    accuracy                           0.67      1200
   macro avg       0.65      0.64      0.64      1200
weighted avg       0.66      0.67      0.66      1200



# HYPERPRAMETER TUNING FOR RANDOMFOREST CLASSIFIER

In [27]:


# Generate a random dataset for demonstration purposes
x, y = make_classification(n_samples=100, n_features=10, n_informative=5, n_classes=2, random_state=42)

# Define the Random Forest classifier
rf = RandomForestClassifier()


In [28]:

# Define the hyperparameters to be tuned
param_grid = {'n_estimators': [100, 200, 300],
              'max_depth': [None, 5, 10],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2, 4]}


In [31]:

# Perform a grid search to find the best hyperparameters
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)
grid_search.fit(x_train_scaled, y_train)


GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 5, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]})

In [32]:

# Print the best hyperparameters and accuracy score
print("Best hyperparameters:", grid_search.best_params_)



Best hyperparameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}


In [33]:
print("Best accuracy score:", grid_search.best_score_)

Best accuracy score: 0.7227574750830564
